# Rally de Datos GTO
## _Semana 4_: Deporte

In [1]:
# Se importan las librerías para el análisis de datos
import pandas as pd
import numpy as np

# Librerías para graficar
import plotly
import plotly.express as px
import plotly.graph_objects as go

# Para hacer lectura de archivos json
from urllib.request import urlopen
import json

# Para usar el algoritmo de clustering de k-means
from sklearn.cluster import KMeans

# Se define la paleta de colores de Dataket 8)
colors = ["#264653", "#2a9d8f", "#e9C46a", "#F4A261", "#E76F51"]

## Lectura de Datasets

In [2]:
# Se define el url donde se encuentra el archivo (en el repositorio de Github)
url_inversion = "https://github.com/GOLF-fisica/Dataket/raw/main/Semana4/Inversion_1T_Qs_por_municipio.csv"

# Se lee el archivo
df_inversion = pd.read_csv(url_inversion)

In [3]:
# El dataset que conecta ODS con Pps
url_ods = "https://github.com/GOLF-fisica/Dataket/raw/main/Semana4/alineacion_pp_ods.csv"

df_ods = pd.read_csv(url_ods, encoding="latin") # El encoding se escoje para que lea las letras con acento

In [4]:
# El dataset propio de la ubicación y población de los municipios de Guanajuato
url_gto = "https://raw.githubusercontent.com/GOLF-fisica/Dataket/main/Semana4/gto_poblacion_ubicacion.csv"

df_gto = pd.read_csv(url_gto)

## Territorialización de la inversión pública asignada en 2021 (actualización al primer trimestre)

In [5]:
# Se convierten los strings del monto a float
df_inversion["Monto"] = df_inversion["Monto"].str.replace(",", "").astype(float)

In [6]:
# Número de proyectos
len(df_inversion["id_proceso_proyecto"].unique())

439

In [7]:
# Número de Pp's
len(df_inversion["id_programa_presupuestario"].unique())

59

In [8]:
# Número de categorías territoriales: 46 municipios + 2 categorías extra
len(df_inversion["Municipio"].unique())

48

In [9]:
# Un df que se modificará para tener una linda gráfica
df_dummy = df_inversion.copy()

# Se añade un directorio base y se coloca el presupuesto en términos de mdp
df_dummy["Programas Presupuestarios"] = "Proyectos de inversión 2021 por Territorio y Programa"
df_dummy["Presupuesto (mdp)"] = round(df_dummy["Monto"].copy()/1_000_000, 2)

# Se hace el treemap
fig = px.treemap(df_dummy,
                 path=["Programas Presupuestarios", "Municipio", "id_programa_presupuestario"],
                 values="Presupuesto (mdp)", color_discrete_sequence=px.colors.sequential.Aggrnyl)

# Se aumenta el tamaño de la letra
fig.update_layout(font_size=13)

# Se guarda el html de la gráfica interactiva en el directorio actual
plotly.offline.plot(fig, filename='/work/treemap_inversion.html')

# Se muestra la gráfica
fig.show()

El archivo _GEO Json_ de los municipios de Guanajuato usado en el siguiente mapa coroplético fue hecho por Alberto Barradas y subido en Diciembre del 2016 al siguiente [enlace](http://datamx.io/sl/dataset/municipios-de-guanajuato).

In [10]:
# Se lee el archivo geojson de los municipios de Guanajuato
url_geojson = "http://datamx.io/sl/dataset/01a8a26c-8ce3-4b4c-a20c-033eeeab1669/resource/b1cd35b7-479e-4fa0-86e9-e897d3c617e6/download/gto.geojson"
with urlopen(url_geojson) as response:
    guanajuato = json.load(response)

# Se agrupa y prepara el dataset de las inversiones al primer trimestre del 2021
df_dummy = df_inversion.copy()

# Se agrupa el presupuesto para cada municipio y se le da el formato de millones
# de pesos, por eso se divide entre un millón
df_choropleth = df_dummy.groupby("Municipio", as_index=False).sum()
df_choropleth["Monto"] = round(df_choropleth["Monto"].copy()/1_000_000, 2)

# Se cambian los nombres de los municipios para que se ajusten a los nombres del geojson
df_choropleth["Municipio"] = df_choropleth["Municipio"].str.replace(" de la Victoria", "")
df_choropleth["Municipio"] = df_choropleth["Municipio"].str.replace("Jérecuaro", "Jerécuaro")


# Se hace el mapa
fig = px.choropleth(df_choropleth, geojson=guanajuato, locations='Municipio',
                    featureidkey="properties.mun_name",
                    color='Monto',
                    color_continuous_scale="Aggrnyl_r", 
                    range_color=(0, 1.5e3), 
                    labels={'Monto':'Presupuesto (mdp)'}
                    )

# Se centra el mapa en Guanajuato
fig.update_geos(fitbounds="locations", visible=False)

# Se configura el margen y el tamaño de la letra
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.update_layout(font_size=15)

# Se guarda el html de la gráfica interactiva
plotly.offline.plot(fig, filename='/work/choropleth.html')

# Se uestra la gráfica
fig.show()

## Los Pp's de los proyectos de inversión asociados al ODS 3

In [11]:
# Los Pp's asociados al ODS 3
df_ods_3 = df_ods[df_ods["id_ods"]==3]
pp_ods_3 = df_ods_3["id_programa_presupuestario"].unique()

In [12]:
# Los Pp's en el ODS 3
pp_ods_3

array(['E064', 'E028', 'E047', 'E014', 'E009', 'E048', 'E012', 'E019'],
      dtype=object)

In [13]:
# Número de Pp's con el ODS 3
len(pp_ods_3)

8

In [14]:
# La inversion en ODS 3
df_inversion_ods_3 = df_inversion[df_inversion["id_programa_presupuestario"].isin(pp_ods_3)]

In [15]:
# El dataset completo con el ODS 3, para lectura fácil
df_ods_3

,id_eje,descripcion_eje,id_programa_presupuestario,descripcion_programa_presupuestario,id_componente,descripcion_componente,id_siglas,descripcion_siglas,id_ods,descripcion_ods,id_meta,descripcion_meta,id_tema_general,descripcion_tema_general,id_tema_objetivo,descripcion_tema_objetivo,id_meta_rubro_propuesta,descripcion_meta_rubro_propuesta
64,2,Desarrollo Humano y Social,E064,Prevención en salud,E064.C04,Acciones de enseñanza pertinente a la transici...,ISAPEG,Instituto de Salud Pública del Estado de Guana...,3,Objetivo 3: Garantizar una vida sana y promove...,3.4,3.4 Reducir en un tercio la mortalidad prematu...,3,Desarrollo social,3.4,Vida sana,3.4.1,"VIH/SIDA, malaria y otras enfermedades"
67,2,Desarrollo Humano y Social,E064,Prevención en salud,E064.C02,Acciones de promoción de estilos de vida salud...,ISAPEG,Instituto de Salud Pública del Estado de Guana...,3,Objetivo 3: Garantizar una vida sana y promove...,3.4,3.4 Reducir en un tercio la mortalidad prematu...,3,Desarrollo social,3.4,Vida sana,3.4.1,"VIH/SIDA, malaria y otras enfermedades"
71,2,Desarrollo Humano y Social,E064,Prevención en salud,E064.C03,Acciones de vigilancia epidemiológica de las e...,ISAPEG,Instituto de Salud Pública del Estado de Guana...,3,Objetivo 3: Garantizar una vida sana y promove...,3.4,3.4 Reducir en un tercio la mortalidad prematu...,3,Desarrollo social,3.4,Vida sana,3.4.1,"VIH/SIDA, malaria y otras enfermedades"
77,2,Desarrollo Humano y Social,E064,Prevención en salud,E064.C01,Acciones preventivas en materia de salud reali...,ISAPEG,Instituto de Salud Pública del Estado de Guana...,3,Objetivo 3: Garantizar una vida sana y promove...,3.4,3.4 Reducir en un tercio la mortalidad prematu...,3,Desarrollo social,3.4,Vida sana,3.4.1,"VIH/SIDA, malaria y otras enfermedades"
78,3,Educación de Calidad,E028,Desarrollo integral de los jóvenes guanajuatenses,E028.C01,Acciones que fomenten la generación de herrami...,JUVENTUDES GTO,Instituto para el Desarrollo y Atención a las ...,3,Objetivo 3: Garantizar una vida sana y promove...,3.4,3.4 Reducir en un tercio la mortalidad prematu...,3,Desarrollo social,3.4,Vida sana,3.4.1,"VIH/SIDA, malaria y otras enfermedades"
97,3,Educación de Calidad,E047,Práctica competitiva y deportiva,E047.C04,Apoyo a deportistas discapacitados del Estado ...,CODE,Comisión Estatal de Cultura Física y Deporte,3,Objetivo 3: Garantizar una vida sana y promove...,3.c,3.c Aumentar sustancialmente la financiación d...,3,Desarrollo social,3.4,Vida sana,3.4.1,"VIH/SIDA, malaria y otras enfermedades"
98,3,Educación de Calidad,E047,Práctica competitiva y deportiva,E047.C07,Apoyo al deporte popular o social otorgado.,CODE,Comisión Estatal de Cultura Física y Deporte,3,Objetivo 3: Garantizar una vida sana y promove...,3.c,3.c Aumentar sustancialmente la financiación d...,3,Desarrollo social,3.4,Vida sana,3.4.1,"VIH/SIDA, malaria y otras enfermedades"
125,3,Educación de Calidad,E047,Práctica competitiva y deportiva,E047.C06,"Asesoría, atención y seguimiento a acciones de...",CODE,Comisión Estatal de Cultura Física y Deporte,3,Objetivo 3: Garantizar una vida sana y promove...,3.c,3.c Aumentar sustancialmente la financiación d...,3,Desarrollo social,3.4,Vida sana,3.4.1,"VIH/SIDA, malaria y otras enfermedades"
128,2,Desarrollo Humano y Social,E014,Certeza jurídica en los procesos conciliatorio...,E014.C01,Atención brindada a quejas e inconformidades p...,CECAMED,Comisión Estatal de Conciliación y Arbitraje M...,3,Objetivo 3: Garantizar una vida sana y promove...,3.8,"3.8 Lograr la cobertura sanitaria universal, i...",3,Desarrollo social,3.4,Vida sana,3.4.1,"VIH/SIDA, malaria y otras enfermedades"
232,3,Educación de Calidad,E047,Práctica competitiva y deportiva,E047.C01,Capacitación al personal en materia deportiva ...,CODE,Comisión Estatal de Cultura Física y Deporte,3,Objetivo 3: Garantizar una vida sana y promove...,3.c,3.c Aumentar sustancialmente la financiación d...,3,Desarrollo social,3.4,Vida sana,3.4.1,"VIH/SIDA, malaria y otras enfermed

In [16]:
# Se define un dataframe que sirva para armar una bonita gráfica
df_dummy = df_inversion_ods_3.groupby("id_programa_presupuestario",
                                      as_index=False).sum().join(df_ods_3.set_index("id_programa_presupuestario")["id_siglas"],
                                                                 on="id_programa_presupuestario").drop_duplicates()

# Se ordenan las filas por monto aprobado y se divide entre un millón y se redondea 
# a dos lugares decimales para que todo quede en formato de millones de pesos
df_dummy = df_dummy.sort_values("Monto", ascending=False)
df_dummy["Monto"] = round(df_dummy["Monto"].copy()/1_000_000, 2)

# Se hace el barplot
fig = px.bar(df_dummy, x="Monto", y="id_programa_presupuestario",
             color="id_siglas", title="Programas presupuestarios asociados al ODS 3",
             color_discrete_sequence=colors,
             labels={'Monto':'Presupuesto (mdp)',
                     "id_programa_presupuestario": "Programa presupuestario",
                     "id_siglas": "Dependencia"})

# Se ordenan las barras por tamaño y se ajusta el tamaño de la letra
fig.update_layout(yaxis={'categoryorder':'total ascending'})
fig.update_layout(font_size=18)

# Se guarda el html en el directorio de trabajo
plotly.offline.plot(fig, filename='/work/barchart_ods_3.html')

# Se muestra la gráfica
fig.show()

In [17]:
# Un df que se modificará para tener una linda gráfica
df_dummy = df_inversion_ods_3.copy()

# Se añade un directorio base y se coloca el presupuesto en términos de mdp
df_dummy["Programas Presupuestarios"] = "Proyectos de inversión 2021 asociados al ODS 3 por Territorio y Programa"
df_dummy["Presupuesto (mdp)"] = round(df_dummy["Monto"].copy()/1_000_000, 2)

# Se hace el treemap
fig = px.treemap(df_dummy,
                 path=["Programas Presupuestarios", "Municipio", "id_programa_presupuestario"],
                 values="Presupuesto (mdp)", color_discrete_sequence=px.colors.sequential.Aggrnyl)

# Se ajusta el tamaño de la letra
fig.update_layout(font_size=13)

# Se guarda el html
plotly.offline.plot(fig, filename='/work/treemap_inversion_ods_3.html')

# Se muestra la gráfica
fig.show()

In [18]:
# Se agrupa y prepara el dataset de las inversiones al primer trimestre del 2021
df_dummy = df_inversion_ods_3.copy()
df_choropleth = df_dummy.groupby("Municipio", as_index=False).sum()
df_choropleth["Monto"] = round(df_choropleth["Monto"].copy()/1_000_000, 2)

# Se cambian los nombres de los municipios para que se ajusten a los nombres del geojson
df_choropleth["Municipio"] = df_choropleth["Municipio"].str.replace(" de la Victoria", "")
df_choropleth["Municipio"] = df_choropleth["Municipio"].str.replace("Jérecuaro", "Jerécuaro")


# Se hace el mapa
fig = px.choropleth(df_choropleth, geojson=guanajuato, locations='Municipio',
                    featureidkey="properties.mun_name",
                    color='Monto',
                    color_continuous_scale="Aggrnyl_r", 
                    range_color=(0, 1.25e2), 
                    labels={'Monto':'Presupuesto (mdp)'}
                    )

# Se centra el mapa en Guanajuato
fig.update_geos(fitbounds="locations", visible=False)

# Se configura el margen
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

fig.update_layout(font_size=15)

plotly.offline.plot(fig, filename='/work/choropleth_ods_3.html')

fig.show()

## Propuesta para dar seguimiento al ODS 3

In [19]:
# A continuación se hará un ejemplo de juguete sobre nuestra propuesta

# Se corrige el tab de los nombres de los municipios
df_gto["Municipio"] = df_gto["Municipio"].str.replace("\t","")

In [20]:
# Se crea la función para implementar un k-means, con el número de clusters que el
# usuario indique y luego coloque los centroides resultantes en una gráfica del
# estado de Guanajuato

def grafica_kmeans(n_clusters):
    # Se prepara el algoritmo
    kmeans = KMeans(n_clusters=n_clusters, n_init=10, random_state=0, max_iter=10_000)

    # Se preparan los arrays
    X = df_gto[["Latitud", "Longitud"]]
    Y = df_gto["Poblacion"]

    # Se hace el clustering usado en X las coordenadas del input y Y es el peso de la muestra
    # en este caso, el peso es la población por cada municipio
    wt_kmeansclus = kmeans.fit(X,sample_weight = Y)
    
    # Los centroides del k-means
    centroides = wt_kmeansclus.cluster_centers_

    # Se hace la gráfica

    # Se prepara el dataset para la gráfica, usando las categorías necesarias
    df_dummy = df_gto[["Municipio", "Longitud", "Latitud", "Poblacion"]].copy()
    df_dummy["Tipo"] = "Municipio"
    df_dummy["Fondo"] = 1
    df_dummy["Municipio"] = df_dummy["Municipio"].str.replace(" de la Victoria", "")
    
    # Se añaden al dataset de la gráffica la ubicación de los centroides del kmeans
    for i in range(len(centroides)):
        df_dummy = df_dummy.append({"Municipio": "Centroide "+str(i+1),
                                    "Longitud": centroides[i][1],
                                    "Latitud": centroides[i][0],
                                    "Tipo": "Centroide"}, ignore_index=True)

    # Se hace el scatter geo con los centroides
    fig = px.scatter_geo(df_dummy[df_dummy["Tipo"]=="Centroide"], # Se van a colocar sólo los centroides
                        lon="Longitud", lat="Latitud",
                        color="Tipo", hover_name="Municipio", fitbounds="locations",
                        color_discrete_sequence=[colors[1],colors[2]], basemap_visible=False)

    # Se hace un mapa coroplético sobre una variable de un solo valor para resaltar
    # la forma del estado. El argumento showscale es para no ostrar la barra de colores
    fig.add_trace(go.Choropleth(geojson=guanajuato, featureidkey="properties.mun_name",
                                locations=df_dummy["Municipio"], z=df_dummy["Fondo"],
                                text=df_dummy["Municipio"],
                                # El hovertemplate es para imprimir sólo los nombres del
                                # municipio en negritas y el segundo string que se muestra
                                # es para que no se imprima qué numero de trace es lo que se
                                # muestra
                                hovertemplate="<b>%{text}</b><br><br>"+"<extra></extra>",
                                showscale=False, colorscale="Viridis",
                                zmin=0, zmax=1.005))

    # Se configura el margen
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

    # Se configura el tamaño de la letra y se quitan las acotaciones
    fig.update_layout(font_size=15)
    fig.update_layout(showlegend=False)

    # Se guarda el html de la gráfica
    plotly.offline.plot(fig, filename='/work/kmeans_'+str(n_clusters)+'.html')

    # Se muestra la gráfica
    fig.show()

In [21]:
# Gráfica para 15 centroides
grafica_kmeans(15)

In [22]:
# Para 20 centroides
grafica_kmeans(20)

In [23]:
# Para 25 centroides
grafica_kmeans(25)

## Presupuesto de inversión territorializado per cápita

In [32]:
df_choropleth_poblacion

,Municipio,ciclo,id_eje,Monto,Poblacion,Monto per capita
0,Abasolo,109134,185,1.356057e+08,92040.0,0.00
1,Acámbaro,129344,208,2.136103e+08,108697.0,0.00
2,Apaseo el Alto,99029,168,6.434133e+07,63392.0,0.00
3,Apaseo el Grande,103071,171,9.148345e+07,117883.0,0.00
4,Atarjea,72756,105,7.117870e+07,5296.0,0.01
5,Celaya,194016,333,4.005625e+08,521169.0,0.00
6,Cobertura Estatal,323360,580,1.914379e+09,NaN,NaN
7,Comonfort,135407,233,9.947362e+07,82216.0,0.00
8,Coroneo,76798,120,3.010314e+07,11083.0,0.00
9,Cortazar,131365,226,1.236020e+08,97928.0,0.00


In [45]:
# Se agrupa y prepara el dataset de las inversiones al primer trimestre del 2021 per capita
df_dummy = df_inversion.copy()

# Se agrupa el presupuesto para cada municipio y se le da el formato de millones
# de pesos, por eso se divide entre un millón
df_choropleth = df_dummy.groupby("Municipio", as_index=False).sum()

# Se cambian los nombres de los municipios para que se ajusten a los nombres del geojson
df_choropleth["Municipio"] = df_choropleth["Municipio"].str.replace("Jérecuaro", "Jerécuaro")

# Se hace el cálculo per cápita
df_choropleth_poblacion = df_choropleth.join(df_gto.set_index("Municipio")["Poblacion"], on="Municipio")
df_choropleth_poblacion["Monto per capita"] = round(df_choropleth_poblacion["Monto"].copy()/(df_choropleth_poblacion["Poblacion"].copy()), 2)

# Se cambian los nombres de los municipios para que se ajusten a los nombres del geojson
df_choropleth_poblacion["Municipio"] = df_choropleth_poblacion["Municipio"].str.replace(" de la Victoria", "")

# Se hace el mapa
fig = px.choropleth(df_choropleth_poblacion, geojson=guanajuato, locations='Municipio',
                    featureidkey="properties.mun_name",
                    color='Monto per capita',
                    color_continuous_scale="tempo", 
                    range_color=(0, 14e3), 
                    labels={'Monto per capita':'Presupuesto per cápita (MXN)'}
                    )

# Se centra el mapa en Guanajuato
fig.update_geos(fitbounds="locations", visible=False)

# Se configura el margen y el tamaño de la letra
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.update_layout(font_size=15)

# Se guarda el html de la gráfica interactiva
plotly.offline.plot(fig, filename='/work/choropleth_per_capita.html')

# Se uestra la gráfica
fig.show()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=8443ea9c-16ed-4729-9b86-cc61a7107837' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>